# Gesture Data Generation

This notebook is designed to capture images of hand gestures from a webcam. The captured images will be used to train a gesture recognition model.

## Configuration

In the next cell, you can configure the parameters for the data capture process.

- `num_screenshots`: The total number of images to capture for the gesture.
- `capture_rate`: The time delay (in seconds) between each screenshot.
- `gesture_name`: The name of the gesture you are capturing (e.g., "point_up", "fist"). This will be used as the folder name and the base name for the image files.
- `folder_location`: The directory where the captured images will be saved. It is automatically set based on the `gesture_name`.

In [ ]:
import os
import time

# --- Configuration ---
num_screenshots = 500  # Number of images to capture
capture_rate = 0.05  # Seconds between captures
gesture_name = "hand_straight"  # Name of the gesture (e.g., "fist", "point_right")

# --- File and Folder Setup ---
base_folder = "data/gestures"
folder_location = os.path.join(base_folder, gesture_name)
picture_name = gesture_name

# Create the directory if it doesn't exist
os.makedirs(folder_location, exist_ok=True)

print(f"Configuration:")
print(f" - Number of screenshots: {num_screenshots}")
print(f" - Capture rate: {capture_rate} seconds")
print(f" - Gesture name: '{gesture_name}'")
print(f" - Saving to: '{folder_location}'")

## Image Capture

Run the following cell to start the image capture process. A window will appear showing your webcam feed.

- **Position your hand** to make the desired gesture.
- Press the **'s'** key to start the automated capture.
- The window will display a countdown of the images being taken.
- Press the **'q'** key at any time to stop the capture and close the window.

In [ ]:
import cv2
import time
import os

cap = cv2.VideoCapture(0)

count = len(os.listdir(folder_location))

screenshots_taken = 0
capturing = False
last_capture_time = 0
window_name = 'Data Generation'

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)
    display_frame = frame.copy()

    if not capturing:
        cv2.putText(display_frame, "Press 's' to start capturing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        cv2.putText(display_frame, f"Capturing... {screenshots_taken}/{num_screenshots}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.putText(display_frame, "Press 'q' to quit", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow(window_name, display_frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('s'):
        if not capturing:
            print("Starting capture...")
            capturing = True
            last_capture_time = time.time()

    if capturing:
        current_time = time.time()
        if current_time - last_capture_time >= capture_rate:
            if screenshots_taken < num_screenshots:
                img_name = os.path.join(folder_location, f"{picture_name}_{count:03d}.jpg")
                cv2.imwrite(img_name, frame)
                print(f"Saved {img_name}")
                count += 1
                screenshots_taken += 1
                last_capture_time = current_time
            else:
                print("Capture complete.")
                break

cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

## Dataset Generation

The following cell processes the images captured in the previous steps. It uses `MediaPipe` to detect hand landmarks in each image, normalizes the landmark data, and then compiles it into a structured dataset.

The final dataset will have two columns:
- `GESTURE_ID`: An integer identifier for each gesture (based on the folder name).
- `LANDMARKS`: The normalized 60-element vector (20 landmarks * 3 coordinates) representing the hand gesture.

This dataset is then saved to a CSV file for use in model training.

In [ ]:
from src.gesture_controller.data_preprocessor import DataPreprocessor
from src.gesture_controller.gesture_detector import GestureDetector
import pandas as pd
import numpy as np
import json
import sys
import cv2
import os


# --- Configuration ---
gestures_folder = "data/gestures"
output_csv_file = "data/gestures_dataset.csv"
key_bindings_file = "config/key_bindings_default.json"

# --- Initialization ---
# We set max_hands to 1 because we are analyzing images of a single gesture
detector = GestureDetector(max_hands=1, min_detection_confidence=0.5)
data_preprocessor = DataPreprocessor()
dataset = []

# --- Data Processing ---
if not os.path.isdir(gestures_folder):
    print(f"Error: Gestures folder not found at '{gestures_folder}'")
else:
    # Get the list of gesture subdirectories and create a mapping to integer IDs
    gesture_folders = sorted([f for f in os.listdir(gestures_folder) if os.path.isdir(os.path.join(gestures_folder, f))])
    gesture_map = {name: i for i, name in enumerate(gesture_folders)}

    print("Processing gestures...")
    print(f"Found gestures: {gesture_map}")

    # Iterate over each gesture folder
    for gesture_name, gesture_id in gesture_map.items():
        folder_path = os.path.join(gestures_folder, gesture_name)
        print(f"\nProcessing folder: {folder_path}")

        # Get a sorted list of images to process them in a consistent order
        image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        # Iterate over each image in the folder
        for image_name in image_files:
            image_path = os.path.join(folder_path, image_name)
            
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Could not read image {image_path}")
                continue

            # Process the frame to find hands
            detector.process_frame(image)
            
            # Get hand vectors (we expect only one hand)
            left_hand, right_hand = detector.get_hand_vectors()
            hand_vector = left_hand or right_hand

            if hand_vector:
                # Normalize the landmarks
                normalized_landmarks = data_preprocessor.process(hand_vector)
                
                if normalized_landmarks is not None:
                    # Append the gesture ID and the flattened list of landmarks
                    dataset.append([gesture_id, normalized_landmarks.tolist()])
            else:
                print(f"Warning: No hand detected in {image_path}")

    # --- Generate Default Key Bindings Configuration ---
    key_bindings_config = {}
    for gesture_name, gesture_id in gesture_map.items():
        key_bindings_config[str(gesture_id)] = {
            "gesture": gesture_name,
            "keys": [],
            "behavior": ""
        }
    
    # Save the default key bindings configuration
    with open(key_bindings_file, 'w') as f:
        json.dump(key_bindings_config, f, indent=2)
    
    # --- Create and Save DataFrame ---
    if dataset:
        # Create a DataFrame from the collected data
        df = pd.DataFrame(dataset, columns=["GESTURE_ID", "LANDMARKS"])
        
        # Save the dataset to a CSV file
        df.to_csv(output_csv_file, index=False)
        
        print(f"\nDataset created successfully with {len(df)} samples.")
        print(f"Saved to '{output_csv_file}'")
        
        # Display the first few rows and the shape of the DataFrame
        print("\nDataset Head:")
        print(df.head())
        print("\nDataset Shape:")
        print(df.shape)
        
    else:
        print("\nNo data was processed. The dataset is empty.")

### Key Bindings Configuration

In addition to the dataset, a `key_bindings_default.json` file is generated. This file maps the detected gestures to specific keyboard actions. You will need to manually edit this file to configure the desired key bindings.

For each gesture, you can specify:
- `keys`: A list of keyboard keys to be triggered. For example: `["ctrl", "c", "up", "right"]`.
- `behavior`: The action to perform with the keys. This can be one of two values:
    - `"press"`: Simulates a quick press and release of the keys (e.g., for typing a character).
    - `"hold"`: Simulates pressing and holding the keys down. The keys will be released when the gesture is no longer detected.